In [149]:
import sys
assert sys.version_info >= (3, 5)
# Python ≥3.5 is required
import sklearn
assert sklearn.__version__ >= "0.20"
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
from pathlib import Path

In [150]:
data = pd.read_csv("03-09-2023.csv")

In [151]:
data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2023-03-10 04:21:03,33.939110,67.709953,209451,7896,NaN,NaN,Afghanistan,538.042451,3.769855
1,NaN,NaN,NaN,Albania,2023-03-10 04:21:03,41.153300,20.168300,334457,3598,NaN,NaN,Albania,11621.968170,1.075774
2,NaN,NaN,NaN,Algeria,2023-03-10 04:21:03,28.033900,1.659600,271496,6881,NaN,NaN,Algeria,619.132366,2.534476
3,NaN,NaN,NaN,Andorra,2023-03-10 04:21:03,42.506300,1.521800,47890,165,NaN,NaN,Andorra,61981.492267,0.344540
4,NaN,NaN,NaN,Angola,2023-03-10 04:21:03,-11.202700,17.873900,105288,1933,NaN,NaN,Angola,320.352770,1.835917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4011,NaN,NaN,NaN,West Bank and Gaza,2023-03-10 04:21:03,31.952200,35.233200,703228,5708,NaN,NaN,West Bank and Gaza,13784.956961,0.811686
4012,NaN,NaN,NaN,Winter Olympics 2022,2023-03-10 04:21:03,39.904200,116.407400,535,0,NaN,NaN,Winter Olympics 2022,NaN,0.000000
4013,NaN,NaN,NaN,Yemen,2023-03-10 04:21:03,15.552727,48.516388,11945,2159,NaN,NaN,Yemen,40.048994,18.074508
4014,NaN,NaN,NaN,Zambia,2023-03-10 04:21:03,-13.133897,27.849332,343135,4057,NaN,NaN,Zambia,1866.491630,1.182333


In [152]:
geoJSONDict = {

  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": []
}


In [153]:
# incident rate = (Cases/population)*100,000
# population = (Cases*100,000)/incident rate
data["Population"] = (data["Confirmed"] * 100000)/data["Incident_Rate"]
data["Cases_per_1000"] = (data["Confirmed"]/data['Population'])*1000
#data["Incident_Rate"] = (data["Deaths"] / data["Population"])* 1000
data["Case_Fatality_Ratio"] = (data["Case_Fatality_Ratio"]*100).round()
data = data[data['Confirmed'] >= data['Deaths']]
data = data.sort_values(by='Incident_Rate', ascending=False)
data.head(50)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population,Cases_per_1000
3495,48301.0,Loving,Texas,US,2023-03-10 04:21:03,31.849476,-103.581857,403,1,NaN,NaN,"Loving, Texas, US",238461.538462,25.0,169.0,2384.615385
755,2068.0,Denali,Alaska,US,2023-03-10 04:21:03,63.672640,-150.007611,1803,2,NaN,NaN,"Denali, Alaska, US",85979.971388,11.0,2097.0,859.799714
765,2180.0,Nome,Alaska,US,2023-03-10 04:21:03,64.903207,-164.035380,7617,7,NaN,NaN,"Nome, Alaska, US",76139.544182,9.0,10004.0,761.395442
767,2188.0,Northwest Arctic,Alaska,US,2023-03-10 04:21:03,67.049192,-159.750395,5794,22,NaN,NaN,"Northwest Arctic, Alaska, US",76026.768141,38.0,7621.0,760.267681
3468,48247.0,Jim Hogg,Texas,US,2023-03-10 04:21:03,27.044539,-98.696819,3805,23,NaN,NaN,"Jim Hogg, Texas, US",73173.076923,60.0,5200.0,731.730769
1106,13053.0,Chattahoochee,Georgia,US,2023-03-10 04:21:03,32.343412,-84.788092,7924,24,NaN,NaN,"Chattahoochee, Georgia, US",72650.591363,30.0,10907.0,726.505914
189,NaN,NaN,Faroe Islands,Denmark,2023-03-10 04:21:03,61.892600,-6.911800,34658,28,NaN,NaN,"Faroe Islands, Denmark",70926.020669,8.0,48865.0,709.260207
750,2050.0,Bethel,Alaska,US,2023-03-10 04:21:03,60.909805,-159.856183,13021,47,NaN,NaN,"Bethel, Alaska, US",70820.189274,36.0,18386.0,708.201893
606,NaN,NaN,NaN,San Marino,2023-03-10 04:21:03,43.942400,12.457800,23616,122,NaN,NaN,San Marino,69585.715128,52.0,33938.0,695.857151
17,NaN,NaN,NaN,Austria,2023-03-10 04:21:03,47.516200,14.550100,5961143,21970,NaN,NaN,Austria,66187.855303,37.0,9006400.0,661.878553


In [154]:
data.loc[data['Country_Region']=='United Arab Emirates']

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population,Cases_per_1000
3986,NaN,NaN,NaN,United Arab Emirates,2023-03-10 04:21:03,23.424076,53.847818,1053213,2349,NaN,NaN,United Arab Emirates,10648.841301,22.0,9890400.0,106.488413


In [155]:
data["Deaths"].sum()

6873116

In [156]:
data["Case_Fatality_Ratio"].sum()

568969.0

In [157]:
data["Incident_Rate"].sum()

111930162.34864745

In [158]:
us_data = data.loc[data['Country_Region']=='US']
data = data[data['Country_Region']!='US']

In [159]:
us_data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population,Cases_per_1000
3495,48301.0,Loving,Texas,US,2023-03-10 04:21:03,31.849476,-103.581857,403,1,NaN,NaN,"Loving, Texas, US",238461.538462,25.0,169.0,2384.615385
755,2068.0,Denali,Alaska,US,2023-03-10 04:21:03,63.672640,-150.007611,1803,2,NaN,NaN,"Denali, Alaska, US",85979.971388,11.0,2097.0,859.799714
765,2180.0,Nome,Alaska,US,2023-03-10 04:21:03,64.903207,-164.035380,7617,7,NaN,NaN,"Nome, Alaska, US",76139.544182,9.0,10004.0,761.395442
767,2188.0,Northwest Arctic,Alaska,US,2023-03-10 04:21:03,67.049192,-159.750395,5794,22,NaN,NaN,"Northwest Arctic, Alaska, US",76026.768141,38.0,7621.0,760.267681
3468,48247.0,Jim Hogg,Texas,US,2023-03-10 04:21:03,27.044539,-98.696819,3805,23,NaN,NaN,"Jim Hogg, Texas, US",73173.076923,60.0,5200.0,731.730769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3753,90051.0,Unassigned,Virginia,US,2020-12-21 13:27:30,NaN,NaN,0,0,NaN,NaN,"Unassigned, Virginia, US",NaN,NaN,NaN,NaN
3798,90053.0,Unassigned,Washington,US,2023-03-10 04:21:03,NaN,NaN,3297,6,NaN,NaN,"Unassigned, Washington, US",NaN,18.0,NaN,NaN
3852,90054.0,Unassigned,West Virginia,US,2023-03-10 04:21:03,NaN,NaN,904,9,NaN,NaN,"Unassigned, West Virginia, US",NaN,100.0,NaN,NaN
3922,90055.0,Unassigned,Wisconsin,US,2023-03-10 04:21:03,NaN,NaN,0,0,NaN,NaN,"Unassigned, Wisconsin, US",NaN,NaN,NaN,NaN


In [160]:
#data struct to store states
# for every state in us_data: 
# get average of Case_Fatality_Ratio, median of lat and long, country
# insert row into data

states =  us_data.Province_State.values.tolist()
states = list(dict.fromkeys(states))
tempStateDf = pd.DataFrame()

for state in states:
    tempDf = us_data.loc[us_data['Province_State']==state]
    stateDf = tempDf.groupby('Province_State', as_index=False).Case_Fatality_Ratio.mean()
    lat = tempDf.groupby('Province_State', as_index=False).Lat.median().iloc[0]['Lat']
    long = tempDf.groupby('Province_State', as_index=False).Long_.median().iloc[0]['Long_']
    cases_per_1000 = tempDf.groupby('Province_State', as_index=False).Cases_per_1000.mean().iloc[0]['Cases_per_1000']
    deaths = tempDf.groupby('Province_State', as_index=False).Deaths.sum().iloc[0]['Deaths']
    confirmed = tempDf.groupby('Province_State', as_index=False).Confirmed.sum().iloc[0]['Confirmed']
    population = tempDf.groupby('Province_State', as_index=False).Population.sum().iloc[0]['Population']
    #  country = "US"
    stateDf["Lat"] = lat
    stateDf["Long_"] = long
    stateDf["Cases_per_1000"] = cases_per_1000
    stateDf["Country_Region"] = 'US'
    stateDf["Deaths"] = deaths
    stateDf["Confirmed"] = confirmed
    stateDf["Population"] = population
    stateDf
    tempStateDf = tempStateDf.append(stateDf, ignore_index=True)
data = data.append(tempStateDf, ignore_index=True)

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\759168216.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempStateDf = tempStateDf.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\759168216.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempStateDf = tempStateDf.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\759168216.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempStateDf = tempStateDf.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\759168216.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempStateDf = tem

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\759168216.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempStateDf = tempStateDf.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\759168216.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempStateDf = tempStateDf.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\759168216.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempStateDf = tempStateDf.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\759168216.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempStateDf = tem

In [161]:
tempStateDf

,Province_State,Case_Fatality_Ratio,Lat,Long_,Cases_per_1000,Country_Region,Deaths,Confirmed,Population
0,Texas,186.419608,31.783209,-98.543035,288.203523,US,93390,8466220,28995881.0
1,Alaska,39.586207,60.067639,-149.953805,457.936320,US,1486,307655,731793.0
2,Georgia,198.863354,32.749765,-83.688450,277.904769,US,42489,3068208,10617423.0
3,Kentucky,126.487603,37.579299,-84.869131,391.886589,US,18130,1718471,4467673.0
4,Colorado,124.400000,38.934415,-105.451273,293.375151,US,14181,1764401,5758736.0
5,South Dakota,165.287879,44.184784,-99.042527,293.472903,US,3190,279130,884659.0
6,Florida,168.073529,29.459336,-82.318310,320.072004,US,86850,7574590,21477737.0
7,North Dakota,114.339623,47.453678,-99.964078,322.700339,US,2232,286950,762062.0
8,Tennessee,141.206186,35.929299,-86.057390,374.344324,US,29263,2515130,6829174.0
9,Wisconsin,99.333333,44.434222,-89.668273,332.086432,US,16375,2006582,5822434.0


In [162]:
data.loc[data['Country_Region']=='US']

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population,Cases_per_1000
733,NaN,NaN,Texas,US,NaN,31.783209,-98.543035,8466220,93390,NaN,NaN,NaN,NaN,186.419608,28995881.0,288.203523
734,NaN,NaN,Alaska,US,NaN,60.067639,-149.953805,307655,1486,NaN,NaN,NaN,NaN,39.586207,731793.0,457.936320
735,NaN,NaN,Georgia,US,NaN,32.749765,-83.688450,3068208,42489,NaN,NaN,NaN,NaN,198.863354,10617423.0,277.904769
736,NaN,NaN,Kentucky,US,NaN,37.579299,-84.869131,1718471,18130,NaN,NaN,NaN,NaN,126.487603,4467673.0,391.886589
737,NaN,NaN,Colorado,US,NaN,38.934415,-105.451273,1764401,14181,NaN,NaN,NaN,NaN,124.400000,5758736.0,293.375151
738,NaN,NaN,South Dakota,US,NaN,44.184784,-99.042527,279130,3190,NaN,NaN,NaN,NaN,165.287879,884659.0,293.472903
739,NaN,NaN,Florida,US,NaN,29.459336,-82.318310,7574590,86850,NaN,NaN,NaN,NaN,168.073529,21477737.0,320.072004
740,NaN,NaN,North Dakota,US,NaN,47.453678,-99.964078,286950,2232,NaN,NaN,NaN,NaN,114.339623,762062.0,322.700339
741,NaN,NaN,Tennessee,US,NaN,35.929299,-86.057390,2515130,29263,NaN,NaN,NaN,NaN,141.206186,6829174.0,374.344324
742,NaN,NaN,Wisconsin,US,NaN,44.434222,-89.668273,2006582,16375,NaN,NaN,NaN,NaN,99.333333,5822434.0,332.086432


In [163]:
country_wise_data = pd.DataFrame()

In [164]:
#data struct to store states
# for every state in us_data: 
# get average of Case_Fatality_Ratio, median of lat and long, country
# insert row into data

countries =  data.Country_Region.values.tolist()
countries = list(dict.fromkeys(countries))

for country in countries:
    tempDf = data.loc[data['Country_Region']==country]
    countryDf = tempDf.groupby('Country_Region', as_index=False).Deaths.sum()
    case_fatality = tempDf.groupby('Country_Region', as_index=False).Case_Fatality_Ratio.median().iloc[0]['Case_Fatality_Ratio']
    lat = tempDf.groupby('Country_Region', as_index=False).Lat.median().iloc[0]['Lat']
    long = tempDf.groupby('Country_Region', as_index=False).Long_.median().iloc[0]['Long_']
  #  country = "US"
    countryDf["Lat"] = lat
    countryDf["Long_"] = long
    countryDf["Case_Fatality_Ratio"] = case_fatality
   # stateDf["Country_Region"] = country
    countryDf
    country_wise_data = country_wise_data.append(countryDf, ignore_index=True)

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

In [165]:
country_wise_data_scaled = country_wise_data
country_wise_data.sort_values(by='Case_Fatality_Ratio', ascending=False).head(100)

,Country_Region,Deaths,Lat,Long_,Case_Fatality_Ratio
196,MS Zaandam,2,NaN,NaN,2222.0
192,Yemen,2159,15.552727,48.516388,1807.0
185,Sudan,5017,12.862800,30.217600,786.0
171,Syria,3164,34.802075,38.996815,551.0
181,Somalia,1361,5.152149,46.199616,498.0
...,...,...,...,...,...
93,Albania,3598,41.153300,20.168300,108.0
70,India,530779,23.780432,78.942140,107.0
139,Venezuela,5854,6.423800,-66.589700,106.0
147,Equatorial Guinea,183,1.650800,10.267900,106.0


In [166]:
country_wise_data.quantile(0.25)

Deaths                 384.000000
Lat                      3.990062
Long_                   -9.128249
Case_Fatality_Ratio     47.750000
Name: 0.25, dtype: float64

In [167]:
country_wise_data.median()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\2100298474.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  country_wise_data.median()


Deaths                 3187.500000
Lat                      17.125350
Long_                    20.921188
Case_Fatality_Ratio     103.000000
dtype: float64

In [168]:
country_wise_data.quantile(0.75).round()

Deaths                 19176.0
Lat                       39.0
Long_                     47.0
Case_Fatality_Ratio      186.0
Name: 0.75, dtype: float64

In [169]:
country_wise_data_scaled["Case_Fatality_Ratio"] = country_wise_data_scaled["Case_Fatality_Ratio"]/2.28
country_wise_data_scaled["Deaths"] = country_wise_data_scaled["Deaths"]/2.28
country_wise_data.sort_values(by='Deaths', ascending=False).tail(100)

,Country_Region,Deaths,Lat,Long_,Case_Fatality_Ratio
171,Syria,1387.719298,34.802075,38.996815,241.666667
186,Nigeria,1383.771930,9.082000,8.675300,51.754386
151,Cambodia,1340.350877,11.550000,104.916700,96.491228
131,Kyrgyzstan,1311.842105,41.204380,74.766098,63.596491
27,Estonia,1292.105263,58.595300,25.013600,21.052632
...,...,...,...,...,...
128,Holy See,0.000000,41.902900,12.453400,0.000000
60,Tuvalu,0.000000,-7.109500,177.649300,0.000000
194,Antarctica,0.000000,-71.949900,23.347000,0.000000
197,Summer Olympics 2020,0.000000,35.649100,139.773700,0.000000


In [170]:
country_wise_data_scaled.quantile(0.25)

Deaths                 168.421053
Lat                      3.990062
Long_                   -9.128249
Case_Fatality_Ratio     20.942982
Name: 0.25, dtype: float64

In [171]:
country_wise_data_scaled.median()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_884\2592085398.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  country_wise_data_scaled.median()


Deaths                 1398.026316
Lat                      17.125350
Long_                    20.921188
Case_Fatality_Ratio      45.175439
dtype: float64

In [172]:
country_wise_data_scaled.quantile(0.75).round()

Deaths                 8410.0
Lat                      39.0
Long_                    47.0
Case_Fatality_Ratio      81.0
Name: 0.75, dtype: float64

In [173]:
clean_data_case_fatality = data.drop(['FIPS', 'Admin2','Province_State','Country_Region','Last_Update','Incident_Rate','Recovered','Active','Combined_Key','Deaths','Confirmed'], axis=1)
clean_data_case_fatality = clean_data_case_fatality.dropna()
clean_data_case_fatality.loc[~(clean_data_case_fatality==0.0).all(axis=1)]
clean_data_case_fatality = clean_data_case_fatality.sort_values(by='Case_Fatality_Ratio', ascending=False)
clean_data_case_fatality

,Lat,Long_,Case_Fatality_Ratio,Population,Cases_per_1000
688,15.552727,48.516388,1807.0,29825968.0,0.400490
675,12.862800,30.217600,786.0,43849269.0,1.455646
604,19.566500,-101.706800,780.0,4825401.0,24.012927
588,19.041400,-98.206300,757.0,6604451.0,33.504526
464,-6.353049,-79.824113,730.0,1310800.0,99.094446
...,...,...,...,...,...
700,35.745200,95.995600,0.0,6030000.0,0.129685
708,32.971100,119.455000,0.0,80510000.0,0.063036
84,51.036200,3.737300,0.0,1515064.0,425.947683
50,-19.054400,-169.867200,0.0,1650.0,480.000000


In [174]:
clean_data_deaths = data.drop(['FIPS', 'Admin2','Province_State','Country_Region','Last_Update','Incident_Rate','Recovered','Active','Combined_Key','Case_Fatality_Ratio','Confirmed'], axis=1)
clean_data_deaths = clean_data_deaths.dropna()
clean_data_deaths.loc[~(clean_data_deaths==0.0).all(axis=1)]
clean_data_deaths = clean_data_deaths.sort_values(by='Deaths', ascending=False)
clean_data_deaths

,Lat,Long_,Deaths,Population,Cases_per_1000
118,52.355500,-1.174300,186138,55977200.0,369.010544
373,-23.550500,-46.633300,179039,45919049.0,140.887979
12,46.227600,2.213700,161512,65273512.0,591.641354
603,-0.789300,113.921300,160941,273523621.0,24.634893
526,19.449759,76.108221,148424,123144223.0,66.086161
...,...,...,...,...,...
581,41.902900,12.453400,0,809.0,35.846724
684,31.692700,88.092400,0,3440000.0,0.478779
697,37.269200,106.165500,0,6880000.0,0.185465
708,32.971100,119.455000,0,80510000.0,0.063036


In [175]:
clean_data_cases_per_1000 = clean_data_deaths.drop(['Deaths','Population'], axis=1)
clean_data_cases_per_1000

,Lat,Long_,Cases_per_1000
118,52.355500,-1.174300,369.010544
373,-23.550500,-46.633300,140.887979
12,46.227600,2.213700,591.641354
603,-0.789300,113.921300,24.634893
526,19.449759,76.108221,66.086161
...,...,...,...
581,41.902900,12.453400,35.846724
684,31.692700,88.092400,0.478779
697,37.269200,106.165500,0.185465
708,32.971100,119.455000,0.063036


In [176]:
#The mistake made here is Im rendering EACH AND EVERY covid case 
#Instead of that, create an object with long, lat and total case count.
#Hence we should only have approx 4000 json objects instead of 600 million+
#TODO: Figure out how mapbox is gonna render it



for i in range(len(clean_data_case_fatality)):
    
    longitude = clean_data_case_fatality.iloc[i]['Long_']
    latitude = clean_data_case_fatality.iloc[i]['Lat']
    data_points = int(clean_data_case_fatality.iloc[i]['Case_Fatality_Ratio'])
    
    
    for x in range(data_points):
        geometry = {
        "geometry": {
          "coordinates": [
            longitude,
            latitude
          ]
        }
      }
        geoJSONDict["features"].append(geometry)
    

json_object = json.dumps(geoJSONDict, separators=(',', ":"))
with open("../GeoJSON/case_fatality_ratio.json", "w") as outfile:
    outfile.write(json_object)

    
geoJSONDict = {

  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": []
}
    
    
    

for i in range(len(clean_data_deaths)):
    
    longitude = clean_data_deaths.iloc[i]['Long_']
    latitude = clean_data_deaths.iloc[i]['Lat']
    data_points = int(clean_data_deaths.iloc[i]['Deaths'])
    
    
    for x in range(data_points):
        geometry = {
        "geometry": {
          "coordinates": [
            longitude,
            latitude
          ]
        }
      }
        geoJSONDict["features"].append(geometry)
    

json_object = json.dumps(geoJSONDict, separators=(',', ":"))
with open("../GeoJSON/deaths.json", "w") as outfile:
    outfile.write(json_object)
    
geoJSONDict = {

  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": []
}
    
    
for i in range(len(clean_data_cases_per_1000)):
    
    longitude = clean_data_case_fatality.iloc[i]['Long_']
    latitude = clean_data_case_fatality.iloc[i]['Lat']
    data_points = int(clean_data_case_fatality.iloc[i]['Cases_per_1000'])
    
    
    for x in range(data_points):
        geometry = {
        "geometry": {
          "coordinates": [
            longitude,
            latitude
          ]
        }
      }
        geoJSONDict["features"].append(geometry)
    

json_object = json.dumps(geoJSONDict, separators=(',', ":"))
with open("../GeoJSON/cases_per_1000.json", "w") as outfile:
    outfile.write(json_object)
    

## Creating the graphs

In [177]:
vaccination_data = pd.read_csv("vaccinations-by-manufacturer.csv")

In [178]:
vaccination_data

,location,date,vaccine,total_vaccinations
0,Argentina,2020-12-29,Oxford/AstraZeneca,1
1,Argentina,2020-12-29,Sinopharm/Beijing,1
2,Argentina,2020-12-29,Sputnik V,20491
3,Argentina,2020-12-30,Sputnik V,40592
4,Argentina,2020-12-31,Sputnik V,43398
...,...,...,...,...
57665,European Union,2023-03-21,Sanofi/GSK,4532
57666,European Union,2023-03-21,Sinopharm/Beijing,2319921
57667,European Union,2023-03-21,Sinovac,8891
57668,European Union,2023-03-21,Sputnik V,1845386


In [179]:
vaccination_data.dropna()

,location,date,vaccine,total_vaccinations
0,Argentina,2020-12-29,Oxford/AstraZeneca,1
1,Argentina,2020-12-29,Sinopharm/Beijing,1
2,Argentina,2020-12-29,Sputnik V,20491
3,Argentina,2020-12-30,Sputnik V,40592
4,Argentina,2020-12-31,Sputnik V,43398
...,...,...,...,...
57665,European Union,2023-03-21,Sanofi/GSK,4532
57666,European Union,2023-03-21,Sinopharm/Beijing,2319921
57667,European Union,2023-03-21,Sinovac,8891
57668,European Union,2023-03-21,Sputnik V,1845386


In [180]:
vaccination_data.drop(['location','date'],axis=1)

,vaccine,total_vaccinations
0,Oxford/AstraZeneca,1
1,Sinopharm/Beijing,1
2,Sputnik V,20491
3,Sputnik V,40592
4,Sputnik V,43398
...,...,...
57665,Sanofi/GSK,4532
57666,Sinopharm/Beijing,2319921
57667,Sinovac,8891
57668,Sputnik V,1845386


In [181]:
vaccines_list = vaccination_data.vaccine.unique().tolist()
vaccine_dict = {}
for i in range(len(vaccines_list)):
    vaccine_dict[vaccines_list[i]] = 0
vaccine_dict

{'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Sputnik V': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'CanSino': 0,
 'Sputnik Light': 0,
 'Johnson&Johnson': 0,
 'Novavax': 0,
 'Sanofi/GSK': 0,
 'Valneva': 0,
 'Medicago': 0,
 'Sinovac': 0,
 'Covaxin': 0,
 'SKYCovione': 0}

In [182]:
vaccine_dict = {'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'Johnson&Johnson': 0,
 'Others':0}
vaccine_dict

{'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'Johnson&Johnson': 0,
 'Others': 0}

In [183]:
for i in range(len(vaccination_data)):
    currVaccine = vaccination_data.iloc[i]['vaccine']
    currCount = vaccination_data.iloc[i]['total_vaccinations']
    
    if currVaccine == 'Oxford/AstraZeneca' or currVaccine == 'Sinopharm/Beijing' or currVaccine == 'Pfizer/BioNTech' or currVaccine == 'Moderna' or currVaccine == 'Johnson&Johnson' :
        dictCount = vaccine_dict[currVaccine]
        vaccine_dict[currVaccine] = int(currCount + dictCount)
    else:
        othersCount = vaccine_dict['Others']
        vaccine_dict['Others'] = int(currCount + dictCount)

In [184]:
vaccine_dict

{'Oxford/AstraZeneca': 82093563042,
 'Sinopharm/Beijing': 31021031484,
 'Pfizer/BioNTech': 939904167926,
 'Moderna': 263357674637,
 'Johnson&Johnson': 26013600382,
 'Others': 31018721338}

In [185]:
json_object = json.dumps(vaccine_dict, separators=(',', ":"))
with open("../mapbox-protype2/src/data/vaccines.json", "w") as outfile:
    outfile.write(json_object)

In [186]:
from datetime import date

totalCases = int(data["Confirmed"].sum())
totalDeaths = int(data["Deaths"].sum())
statsDict ={
    'totalCases': f"{totalCases:,}",
    'totalDeaths':f"{totalDeaths:,}",
    'cases_per_1000':int((totalCases/data["Population"].sum())*1000),
    'lastUpdated': date.today().strftime("%d/%m/%Y")
}
statsDict

{'totalCases': '676,569,930',
 'totalDeaths': '6,873,116',
 'cases_per_1000': 87,
 'lastUpdated': '20/04/2023'}

In [187]:
json_object = json.dumps(statsDict, separators=(',', ":"))
with open("../mapbox-protype2/src/data/stats.json", "w") as outfile:
    outfile.write(json_object)

In [188]:
data['Cases_per_1000'].sum()

153675.8368342452

In [189]:
clean_data_cases_per_1000.quantile(0.25)

Lat                8.778100
Long_            -68.641025
Cases_per_1000    68.682005
Name: 0.25, dtype: float64

In [190]:
clean_data_cases_per_1000.median()

Lat                33.712259
Long_              14.662780
Cases_per_1000    164.105972
dtype: float64

In [191]:
clean_data_cases_per_1000.quantile(0.75).round()

Lat                47.0
Long_              73.0
Cases_per_1000    298.0
Name: 0.75, dtype: float64

In [192]:
clean_data_case_fatality.quantile(0.25)

Lat                         8.778100
Long_                     -68.641025
Case_Fatality_Ratio        47.000000
Population             786207.500000
Cases_per_1000             68.682005
Name: 0.25, dtype: float64

In [193]:
clean_data_case_fatality.median().round()

Lat                         34.0
Long_                       15.0
Case_Fatality_Ratio        106.0
Population             1944794.0
Cases_per_1000             164.0
dtype: float64

In [194]:
clean_data_case_fatality.quantile(0.75).round()

Lat                         47.0
Long_                       73.0
Case_Fatality_Ratio        192.0
Population             6839702.0
Cases_per_1000             298.0
Name: 0.75, dtype: float64

In [195]:
clean_data_deaths.quantile(0.25).round()

Lat                    9.0
Long_                -69.0
Deaths               630.0
Population        786208.0
Cases_per_1000        69.0
Name: 0.25, dtype: float64

In [196]:
clean_data_deaths.median().round()

Lat                    34.0
Long_                  15.0
Deaths               2706.0
Population        1944794.0
Cases_per_1000        164.0
dtype: float64

In [197]:
clean_data_deaths.quantile(0.75).round()

Lat                    47.0
Long_                  73.0
Deaths               8387.0
Population        6839702.0
Cases_per_1000        298.0
Name: 0.75, dtype: float64